# Step 1 - Importing of relevant functions

In [1]:
import os
import sys
sys.path.append(os.path.relpath('code' ))

import numpy as np
import scipy
import copy
import scipy.optimize as opt
import matplotlib.pyplot as plt
from Method_functions import *
from Field_functions import *
import pickle 
from pathlib import Path 


## Variation of parameters for the paper
Here are the variables that are changed for the paper figures

**Vary parameters here for creating the data for the different figures**

In [2]:
#########################################################################################################
#########################################################################################################

step_size = 0.1  # best to choose h=[0.01,0.02,0.04,0.08,0.1,0.2, 0.4, 0.8]  as for these are divisors of 28 and thus all numerical methods have the same end time

alpha_value,beta_value,gamma_value=1,1,1
# alpha_value,beta_value,gamma_value=1,0,0
# alpha_value,beta_value,gamma_value=0.5,0.5,0.5


# start guess that is close to the optimal control and thus allows for convergence to proper solution of root finding method
# for large step-sizes, like 0.8 and first-order schemes it might be better to use e.g. 0.4, 0.2 as the initial guesses, as the default one is for a method that has converged to the real solution
save_data_file = 'code/start_guess_traj.pkl'


# set to False if one is interested in also getting the optimal solution of the control dependent case, for fig. 1 relevant
skip_control_dependent_solution_search = False
# set to False if one is interested to also generate the solution via the direct method, for fig 2 relevant
skip_direct_minimization  = True

#########################################################################################################
#########################################################################################################


# Step 2 - Give the parameters needed for the problem

In [3]:
###########
# Dict that contains all parameters
###########

parameters = dict({ "M" : 10., 
                   "t_0" : 0, "t_N" : 3, 
                   "q_0" : np.array([[4.],[0]]), "q_T" : np.array([[-5.],[0]]), 
                   "dim_u" : 1,
                   "h":step_size,
                   "N": 0,"alpha":alpha_value, 
                   "beta":beta_value,"gamma":gamma_value,
                   "grav" : 1, #grav=gravitational constant
                   "q_T_weight": 1, "dq_T_weight": 1,
                   "times":np.array([0]),
                   "d" :1 #number rotations around
                   }) 

parameters["dim_q"] = len(parameters['q_0'])
# parameters["t_N"] = 1.5*end_Time_calc(parameters["d"],parameters["M"],parameters["grav"],np.sqrt(parameters["q_0"].flatten()@parameters["q_0"].flatten()), np.sqrt(parameters["q_T"].flatten()@parameters["q_T"].flatten()))
parameters["t_N"] = 28  #Choose rounded time here, which is approximately the time value as calculated by the function in the paper. However this rounding enables one to discretize such that the total time is always fix, for the h-choices
parameters["dq_T"] = np.array([[0],[dphi_from_radius(abs(parameters["q_T"][0,0]),parameters["grav"],parameters["M"])]])
parameters["dq_T"] = np.array([[0],[-abs(parameters["q_T"][0][0])*parameters["dq_T"][1][0]]])
parameters["dq_0"] = np.array([[0],[dphi_from_radius(parameters["q_0"][0,0],parameters["grav"],parameters["M"])]])
parameters["dq_0"] = np.array([[0],[parameters["q_0"][0][0]*parameters["dq_0"][1][0]]])
rescale_stepsize_parameters(parameters,parameters["h"]) 



# Step 3 - Generate good starting values for efficient convergence

In [4]:
# Create start-guess for simulation


with open(save_data_file, 'rb') as files:
    initial_guess_data = pickle.load(files)

q_d_start_guess = np.array(initial_guess_data['q_d'])
lam_d_start_guess =np.array(initial_guess_data['lam_d'])
u_d_start_guess = initial_guess_data['U_d']

cs_q =  scipy.interpolate.CubicSpline(np.linspace(0,parameters["t_N"],len(q_d_start_guess)), q_d_start_guess.reshape([len(q_d_start_guess),2]))
cs_lam = scipy.interpolate.CubicSpline(np.linspace(0,parameters["t_N"],len(lam_d_start_guess)), lam_d_start_guess.reshape([len(lam_d_start_guess),2]))  
cs_u= scipy.interpolate.CubicSpline(np.linspace(0,parameters["t_N"],len(u_d_start_guess)),u_d_start_guess)

U_d_1_use = cs_u(np.array(parameters["times"]) + parameters["beta"]*parameters["h"]).reshape(len(parameters["times"]),1,1)
U_d_2_use = cs_u(np.array(parameters["times"]) + (1-parameters["beta"])*parameters["h"]).reshape(len(parameters["times"]),1,1)

q_d_use = cs_q(parameters["times"]).reshape(len(parameters["times"]),2,1)
lambda_d_use = cs_lam(parameters["times"]).reshape(len(parameters["times"]),2,1)
mu_use = initial_guess_data['mu']
nu_use = initial_guess_data['nu']

 
 #can plot here the trajectories to see what they look like
# plot_curve(q_d_use,"example_trajectory",parameters);
# plt.show()
# plt.plot(np.array(parameters["times"]) + parameters["beta"]*parameters["h"], U_d_1_use.flatten())
# plt.plot(np.array(parameters["times"]) + (1-parameters["beta"])*parameters["h"], U_d_2_use.flatten())

# Step 4.1 - Create solution for the u-independent Lagrangian

In [10]:
# no-u dependent case

stacked_startvals = stack_variables_two_Us(q_d_use,lambda_d_use,mu_use,nu_use,U_d_1_use,U_d_2_use,False)
optimality_conditions = optimality_conditions_generator(vector_field_eval,covector_field_eval,parameters)
optifunc = lambda X: np.concatenate(optimality_conditions.create_all_optimality_conditions_no_u(*unstack_variables_two_Us(X,parameters["dim_q"],parameters["dim_u"],parameters["N"],False)[:-2],U_d_1_use,U_d_2_use))

solution_no_u = opt.root(fun=optifunc,x0=stacked_startvals,method='lm')
#Alternative root finding method, sometimes faster, but typically not as accurate:
# solution_no_u = opt.root(fun=optifunc,x0=stacked_startvals,method='hybr')

#print solution message
solution_no_u

# Step 4.2 - Create solution for the u-dependent Lagrangian

Typically will be slower than the independent case by virtue of having more variables (the explicit treatment of the controls)

Can be skipped if not considered, e.g. for the generation of fig. 3

Will be skipped via setting the flag 'skip_control_dependent_solution_search' to True 

In [273]:
#run the simulation with the startvalues from the control independent case, as they should lead to the same solutions
# that typically guarantees fast convergence

if not skip_control_dependent_solution_search:
    q_d_use,lambda_d_use,mu_use,nu_use  = unstack_variables_two_Us(solution_no_u.x,parameters["dim_q"],parameters["dim_u"],parameters["N"],False)[:-2]
    U_d_1_use = np.array([get_u_from_lambda(x,y,parameters) for (x,y) in zip(lambda_d_use,q_d_use)])
    U_d_2_use = np.array([get_u_from_lambda(x,y,parameters) for (x,y) in zip(lambda_d_use,q_d_use)])

    stacked_startvals = stack_variables_two_Us(q_d_use,lambda_d_use,mu_use,nu_use,U_d_1_use,U_d_2_use)

    dim_q = len(q_d_use[0])
    dim_u = len(U_d_1_use[0])

    optimality_conditions = optimality_conditions_generator(vector_field_u_eval,covector_field_u_eval,parameters)
    optifunc = lambda X: np.concatenate(optimality_conditions.create_all_optimality_conditions(*unstack_variables_two_Us(X,dim_q,dim_u,parameters["N"])))

    solution_u = opt.root(fun=optifunc,x0=stacked_startvals,method='lm')
    solution_u


Sometimes it can make sense to run the root finder again, due to U_d_1, U_d_2 being sometimes the same variables, resulting in a highly deficit jacobian 
e.g. for alpha=beta=gamma = 1. 

This makes the convergence to the solution sometimes unstable. Redoing the root finding with the prior solution helps there

In [1]:
#quick check that can be done to see whether the root finder was able to properly find a solution. If no solution is found, then typically the conserved quantity is not conserved
# q_d_sol_test, lam_d_sol_test, mu_test,nu_test,u_d_1_test = unstack_variables(solution_no_u.x,dim_q,dim_u, parameters["N"],False) 
# from Analysis_helper_functions import calculate_conserved_quantity_evolution
# conserved_I_evo = calculate_conserved_quantity_evolution(q_d_sol_test,lam_d_sol_test,u_d_1_test,u_d_1_test,vector_field_eval,covector_field_eval,parameters)

# plt.plot(conserved_I_evo,'--*')

# Step 5 - Save solutions

In [291]:
if not solution_no_u.success:
        print('did not find a solution in control dependent case, not storing the result')
else:
    storage_dict = dict()
    storage_dict["q_d"],storage_dict["lam_d"],storage_dict["mu"],storage_dict["nu"],storage_dict["U_d"]  = unstack_variables(solution_no_u.x,parameters["dim_q"],parameters["dim_u"],parameters["N"],False)
    storage_dict["U_d"]  = [get_u_from_lambda(x,y,parameters) for (x,y) in zip(storage_dict["lam_d"],storage_dict["q_d"])]
    storage_dict["parameters"] = parameters
    storage_dict["solution_info"] = solution_no_u
    dirpath = "data/no_u_dep_data_"+"a=" + str(parameters["alpha"]) + "b=" + str(parameters["beta"]) +"g=" + str(parameters["gamma"])
    Path(dirpath).mkdir(parents=True, exist_ok=True)

    no_u_dep_file_name = dirpath+"/no_u_dep_data_" +"a=" + str(parameters["alpha"]) + "b=" + str(parameters["beta"]) +"g=" + str(parameters["gamma"])  +"h=" + str(parameters["h"]) + ".pkl"
    with open(no_u_dep_file_name, 'wb') as ffile:
        pickle.dump(storage_dict, ffile)
        ffile.close()
    
if not skip_control_dependent_solution_search:
    if not solution_u.success:
        print('did not find a solution in control dependent case, not storing the result')
    else:
        storage_dict = dict()
        storage_dict["q_d"],storage_dict["lam_d"],storage_dict["mu"],storage_dict["nu"],storage_dict["U_d_1"], storage_dict["U_d_2"]  = unstack_variables_two_Us(solution_u.x,parameters["dim_q"],parameters["dim_u"],parameters["N"])
        storage_dict["parameters"] = parameters
        storage_dict["solution_info"] = solution_u

        dirpath = "data/new_u_dep_data_"+"a=" + str(parameters["alpha"]) + "b=" + str(parameters["beta"]) +"g=" + str(parameters["gamma"])
        Path(dirpath).mkdir(parents=True, exist_ok=True)
        u_dep_file_name = dirpath+"/new_u_dep_data_" +"a=" + str(parameters["alpha"]) + "b=" + str(parameters["beta"]) +"g=" + str(parameters["gamma"])   +"h=" + str(parameters["h"]) + ".pkl"
        with open(u_dep_file_name, 'wb') as ffile:
            pickle.dump(storage_dict, ffile)
            ffile.close()





# Standard direct approach comparison

In [47]:
# standard approach calc
if not skip_direct_minimization:
    optimality_conditions = optimality_conditions_generator(vector_field_u_eval,covector_field_u_eval,parameters)
    q_d_use1, lam_d_use1, mu_use1, nu_use1,_,_ = unstack_variables_two_Us(solution_no_u.x,dim_q,dim_u,parameters["N"],False)
    # q_d_use1, lam_d_use1, mu_use1, nu_use1,U_d_use,_ = unstack_variables_two_Us(solution_u.x,dim_q,dim_u,parameters["N"])

    controls_nodes = np.array([get_u_from_lambda(x,y,parameters) for (x,y) in zip(lam_d_use1,q_d_use1)]).flatten()
    midpoint_controls_startguess = np.array([(controls_nodes[i]+controls_nodes[i+1])/2.0 for i in range(len(controls_nodes)-1) ])
    # solution_traj = optimality_conditions.standard_create_running_termina_cost(midpoint_controls)
    cost_function=lambda X : sum(optimality_conditions.standard_create_running_termina_cost(np.reshape(X,[len(X),1,1]))[0])

    minimized_sol = opt.minimize(cost_function,np.array(midpoint_controls_startguess).flatten(),method='SLSQP')
    u_vals_standard = np.reshape(minimized_sol.x,[parameters["N"]+1,1,1])
    solvals,q_vals_standard,v_vals_standard = optimality_conditions.standard_create_running_termina_cost(u_vals_standard)
    if not minimized_sol.success:
        print('did not find a solution in control dependent case, not storing the result')
    else:
        storage_dict = dict()
        storage_dict["q_d"],storage_dict["v_d"],storage_dict["U_d"]  = q_vals_standard,v_vals_standard,u_vals_standard
        storage_dict["parameters"] = parameters
        storage_dict["minimizer_info"] = minimized_sol
        dirpath = "data/standard_comparison_midpoint"
        Path(dirpath).mkdir(parents=True, exist_ok=True)

        standard_file_name = dirpath+"/standard_comparison_midpoint_"   +"h=" + str(parameters["h"]) + ".pkl"

        with open(standard_file_name, 'wb') as ffile:
            pickle.dump(storage_dict, ffile)
            ffile.close()

#can also plot here the curve if interested

# plot_curve(q_vals_standard,'test_minimizer',parameters)
# plt.show()
# plt.plot(parameters["times"][:-1],(u_vals_standard.flatten()[:-1]+u_vals_standard.flatten()[1:])/2,label='standard')
# plt.plot(initial_guess_data_midpoint['parameters']['times'][:-1],(np.array(initial_guess_data_midpoint['U_d']).flatten()[1:]+np.array(initial_guess_data_midpoint['U_d']).flatten()[:-1])/2 ,label='midpoint_new_sol')
# plt.legend()
# plt.plot((u_vals_standard.flatten()))
# plt.ylim([-0.001,0.005])